<left>
    <h3>Data Preparation Model kedua</h3>
</left>

---

<h3>STEP 1 - DATA  COLLECTION AND LOADING</h3>

In [2]:
import pandas as pd

file_food = "Dataset/foods.csv"
df_food = pd.read_csv(file_food, sep=";")
print("✅ Data makanan:")
print(df_food.head(), "\n")

✅ Data makanan:
     FoodItems  Energy(kcal)  Carbs  Protein(g)  Fat(g)  Freesugar(g)  \
0   Butternaan         300.0   50.0         7.0    10.0           2.0   
1      Cupcake         200.0   30.0         2.0     8.0          20.0   
2       Donuts         250.0   30.0         3.0    12.0          10.0   
3  FrenchFries         312.0   41.0         3.4    15.0           0.3   
4  GarlicBread         200.0   25.0         4.0    10.0           1.0   

   Fibre(g)  cholestrol(mg)  calcium(mg)  
0       2.0            15.0         50.0  
1       0.5            20.0         20.0  
2       1.0            20.0         20.0  
3       3.8             0.0         20.0  
4       1.0            10.0         30.0   



 <h3>STEP 2 - DATA CLEANSING & PREPROCESSING</h3>

In [12]:
df_bersih = df_food.dropna(how='any', axis=0)
output_path = "C:/Users/ASUS/Machine Learning project/Diabetes_MealPlan_Model/Dataset/food_clean.csv"
df_bersih.to_csv(output_path, index=False)

print(f"File gabungan berhasil dibuat: {output_path}")

File gabungan berhasil dibuat: C:/Users/ASUS/Machine Learning project/Diabetes_MealPlan_Model/Dataset/food_clean.csv


In [13]:
df_bersih.dropna(how='any', axis=0)

,FoodItems,Energy(kcal),Carbs,Protein(g),Fat(g),Freesugar(g),Fibre(g),cholestrol(mg),calcium(mg)
0,Butternaan,300.00,50.00,7.00,10.00,2.00,2.00,15.0,50.00
1,Cupcake,200.00,30.00,2.00,8.00,20.00,0.50,20.0,20.00
2,Donuts,250.00,30.00,3.00,12.00,10.00,1.00,20.0,20.00
3,FrenchFries,312.00,41.00,3.40,15.00,0.30,3.80,0.0,20.00
4,GarlicBread,200.00,25.00,4.00,10.00,1.00,1.00,10.0,30.00
...,...,...,...,...,...,...,...,...,...
1023,Sweetandsourtomatopickle(Khattameethatamatarka...,60.88,6.55,1.26,3.24,4.31,2.20,0.0,15.18
1024,Jhatpatacharwithcarrot(Jhatpatachaargajarkesaath),91.21,6.32,1.98,6.55,3.04,5.08,0.0,54.31
1025,Tomatochutney(Tamatarkichutney),176.07,31.85,0.97,6.01,30.02,1.49,0.0,25.34
1026,Tomatoketchup,33.07,6.48,0.91,0.30,4.68,1.90,0.0,15.33


In [23]:
import pandas as pd

df_bersih = pd.read_csv("Dataset/food_clean.csv")

# --- 0. Normalisasi nama kolom ---
df_bersih.columns = (
    df_bersih.columns.str.lower()
              .str.replace(r'[^a-z0-9]+', '_', regex=True)
              .str.strip('_')
)

print("Kolom setelah normalisasi:", df_bersih.columns.tolist())

# --- 1. Deteksi kolom nama makanan (kolom string pertama) ---
object_cols = df_bersih.select_dtypes(include=['object']).columns.tolist()

if len(object_cols) == 0:
    raise ValueError("Tidak ada kolom string. Nama makanan mungkin berubah menjadi numeric. STOP!")

name_col = object_cols[0]
print("Kolom makanan terdeteksi sebagai:", name_col)

# --- 2. Kolom nutrisi = semua kecuali nama makanan ---
numeric_cols = [col for col in df_bersih.columns if col != name_col]

# --- 3. Ganti koma menjadi titik hanya pada kolom numerik ---
df_bersih[numeric_cols] = df_bersih[numeric_cols].replace(",", ".", regex=True)

# --- 4. Imputasi ---
fill_zero = ["protein_g", "freesugar_g", "fibre_g"]
fill_median = ["energy_kcal", "carbs", "fat_g", "cholestrol_mg", "calcium_mg"]

# isi zero
for col in fill_zero:
    if col in df_bersih.columns:
        df_bersih[col] = pd.to_numeric(df_bersih[col], errors='coerce').fillna(0)

# isi median
for col in fill_median:
    if col in df_bersih.columns:
        df_bersih[col] = pd.to_numeric(df_bersih[col], errors='coerce').fillna(df_bersih[col].median())

# --- 5. Convert semua numeric ---
for col in numeric_cols:
    df_bersih[col] = pd.to_numeric(df_bersih[col], errors='coerce')

# --- 6. HAPUS DUPLIKAT (TAPI TIDAK MENGGANGGU KOLOM NAMA MAKANAN) ---
df_bersih.drop_duplicates(inplace=True)

# --- 7. SIMPAN ---
df_bersih.to_csv("foods_prepared.csv", index=False)

print("\nSelesai! Nama makanan aman dan file tersimpan sebagai foods_prepared.csv")


Kolom setelah normalisasi: ['fooditems', 'energy_kcal', 'carbs', 'protein_g', 'fat_g', 'freesugar_g', 'fibre_g', 'cholestrol_mg', 'calcium_mg']
Kolom makanan terdeteksi sebagai: fooditems

Selesai! Nama makanan aman dan file tersimpan sebagai foods_prepared.csv
